In [22]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts.prompt import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage

In [3]:
class Utf8TextLoader(TextLoader):
    def __init__(self, *args, **kwargs):
        kwargs['encoding'] = 'utf-8'
        super().__init__(*args, **kwargs)

loader = DirectoryLoader(
    "./DATA", glob="**/*.txt", loader_cls=Utf8TextLoader, show_progress=True
)
docs = loader.load()

100%|██████████| 5/5 [00:00<00:00, 1666.79it/s]


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(docs)
len(chunks)

33

In [9]:
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma.from_documents(chunks, embedding_function)
retriever = db.as_retriever()

d:\Repos\Energy-IA-Server-Ollama-Mistral\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Repos\Energy-IA-Server-Ollama-Mistral\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
retriever.invoke("Cuáles son los costos de incorporación")

[Document(metadata={'source': 'DATA\\preguntas.txt'}, page_content='¿Cuáles son los costos de incorporación y afiliación al Servicio de Bienestar? Cuota de incorporación, que corresponde a un 2% del sueldo imponible o hasta el tope imponible del mes, se descuenta sólo en el primer mes de ingreso. \n2. Aporte mensual, que corresponde a un 1,7% del sueldo imponible o hasta el tope imponible del mes.\u200b\u200b'),
 Document(metadata={'source': 'DATA\\contacto.txt'}, page_content='\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b¿Cómo incorporarse?\nDebes poseer la calidad jurídica de planta, contrata o suplencia, y rellenar el formulario de incorporación. Luego lo debes enviar bienestar@minenergia.cl \n\nBajos Costos\nCuota de incorporación: 0.7% del imponible (se paga por única vez al momento de ingresar).\nCuota de descuento mensual: 1,7% del imponible.\n\ncontacto \nRomina Vallejos Gallardo - Encargada de Bienestar\nbienestar@

In [17]:
rephrase_template = """Dada la siguiente conversación y una pregunta de seguimiento, reformule la pregunta de seguimiento para que sea una pregunta independiente, en su idioma original..

Historial de chat:
{chat_history}
Entrada de seguimiento: {question}
Pregunta independiente:"""

REPHRASE_TEMPLATE = PromptTemplate.from_template(rephrase_template)
rephrase_chain = REPHRASE_TEMPLATE | ChatOllama(model="mistral",temperature=0) | StrOutputParser()

template = """Como un asistente del departamento de Bienestar del Ministerio de energia, 
Responda la pregunta lo mas completa posible basándose únicamente en el siguiente contexto.:

{context}

Pregunta: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [20]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | ChatOllama(model="mistral",temperature=0)
    | StrOutputParser()
)

final_chain = rephrase_chain | retrieval_chain

In [21]:
final_chain.invoke(
    {
        "question": "como se envian los gastos medicos",
        "chat_history": [
           
        ],
    }
)

'1. Primero, es necesario ir al médico o dentista y llevar el formulario del seguro que está disponible en la sección Formularios. El médico deberá llenar los datos solicitados, como por ejemplo: el diagnóstico por la atención y los procedimientos asociados al diagnóstico.\n\n2. En situaciones especiales no es necesario que el formulario sea llenado por el médico, como en el caso de una continuación de un tratamiento o una atención de urgencia.\n\n3. Detalles sobre los porcentajes de reembolso según la prestación médica realizada no se proporcionan en este contexto.\n\n4. Algunas prestaciones como consultas médicas y exámenes, la cobertura aplica automáticamente por I-med (no siempre, hay que fijarse en el bono), por eso se recomienda siempre andar con el formulario, para solicitar el reembolso en caso de que I-med no lo realice en línea.\n\n5. No se proporciona información sobre cómo se realizan las transferencias de gastos médicos en este contexto. Es posible que sea necesario contac

In [23]:
final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

' Based on the provided context, there is no information about "Thuna" being a type of food for dogs. The documents contain information about a complementary insurance package offered by the Ministry of Energy, which includes life, health, catastrophic, and dental insurance for employees and their dependents. The cost, coverage details, and deductibles for each type of insurance are also provided. Additionally, it mentions that employees may access various benefits with external companies such as discounts on dental clinics, hair salons, yoga classes, etc., and participate in activities organized by the Unidad de Bienestar y Calidad de Vida (Unit for Well-being and Quality of Life). However, there is no mention of "Thuna" being related to dogs or dog food.'